In [17]:
#%pip install statsforecast pandas polars pyarrow

In [18]:
import polars as pl
import pandas as pd

from statsforecast import StatsForecast
from statsforecast.models import SeasonalNaive, HistoricAverage, RandomWalkWithDrift
from statsforecast.models import SimpleExponentialSmoothing, Holt, HoltWinters, AutoRegressive, AutoARIMA

In [19]:
electricity_df = pl.read_csv("../data/eu_electricity_daily.csv")
electricity_df = electricity_df.with_columns(pl.col("Date").str.strptime(pl.Date, format="%Y-%m-%d").alias("Date"))

### Only use data for austria:

In [20]:
electricity_df = electricity_df.filter(electricity_df["ISO3 Code"] == "AUT")
electricity_df


Country,ISO3 Code,Date,Price (EUR/MWhe)
str,str,date,f64
"""Austria""","""AUT""",2015-01-01,22.34
"""Austria""","""AUT""",2015-01-02,22.34
"""Austria""","""AUT""",2015-01-03,22.34
"""Austria""","""AUT""",2015-01-04,22.34
"""Austria""","""AUT""",2015-01-05,36.18
…,…,…,…
"""Austria""","""AUT""",2025-12-18,135.16
"""Austria""","""AUT""",2025-12-19,126.37
"""Austria""","""AUT""",2025-12-20,111.08


In [21]:
electricity_df.null_count() # Data contains no null values

Country,ISO3 Code,Date,Price (EUR/MWhe)
u32,u32,u32,u32
0,0,0,0


### Create daily dataset:

In [22]:
daily_df = electricity_df.select([
    pl.col("Country").alias("unique_id"),
    pl.col("Date").alias("ds"),
    pl.col("Price (EUR/MWhe)").alias("y")
]).sort("ds")
daily_df

unique_id,ds,y
str,date,f64
"""Austria""",2015-01-01,22.34
"""Austria""",2015-01-02,22.34
"""Austria""",2015-01-03,22.34
"""Austria""",2015-01-04,22.34
"""Austria""",2015-01-05,36.18
…,…,…
"""Austria""",2025-12-18,135.16
"""Austria""",2025-12-19,126.37
"""Austria""",2025-12-20,111.08


### Create montly dataset:

In [23]:
monthly_df = electricity_df.with_columns(pl.col("Date").dt.truncate("1mo").alias("month_first"))
monthly_df

Country,ISO3 Code,Date,Price (EUR/MWhe),month_first
str,str,date,f64,date
"""Austria""","""AUT""",2015-01-01,22.34,2015-01-01
"""Austria""","""AUT""",2015-01-02,22.34,2015-01-01
"""Austria""","""AUT""",2015-01-03,22.34,2015-01-01
"""Austria""","""AUT""",2015-01-04,22.34,2015-01-01
"""Austria""","""AUT""",2015-01-05,36.18,2015-01-01
…,…,…,…,…
"""Austria""","""AUT""",2025-12-18,135.16,2025-12-01
"""Austria""","""AUT""",2025-12-19,126.37,2025-12-01
"""Austria""","""AUT""",2025-12-20,111.08,2025-12-01


In [24]:
monthly_df = monthly_df.group_by(["Country", "month_first"]).agg(pl.col("Price (EUR/MWhe)").mean().alias("mean_price")).sort("month_first")

In [25]:
monthly_df = monthly_df.rename({
    "Country": "unique_id",
    "month_first": "ds",
    "mean_price":"y"
})
monthly_df

unique_id,ds,y
str,date,f64
"""Austria""",2015-01-01,29.935161
"""Austria""",2015-02-01,36.695
"""Austria""",2015-03-01,31.297419
"""Austria""",2015-04-01,29.778333
"""Austria""",2015-05-01,25.329677
…,…,…
"""Austria""",2025-08-01,74.744194
"""Austria""",2025-09-01,92.587667
"""Austria""",2025-10-01,108.226452


### Split datasets:

In [26]:
daily_df = daily_df.to_pandas()
monthly_df = monthly_df.to_pandas()

In [27]:
daily_train = daily_df[daily_df["ds"] < pd.to_datetime("2025-01-01")]
daily_val = daily_df[(daily_df["ds"] >= pd.to_datetime("2025-01-01")) & (daily_df["ds"] < pd.to_datetime("2026-01-01"))]
daily_test = None

In [28]:
monthly_train = monthly_df[monthly_df["ds"] < pd.to_datetime("2025-01-01")]
monthly_val = monthly_df[(monthly_df["ds"] >= pd.to_datetime("2025-01-01")) & (monthly_df["ds"] < pd.to_datetime("2026-01-01"))]
monthly_test = None

### Forecasts:

In [29]:
HORIZON_DAILY = 365 # days
HORIZON_MONTHLY = 12 # months

##### Baseline Forecast:

In [ ]:
sf_daily = StatsForecast(
    models=[
        SeasonalNaive(season_length=HORIZON_DAILY),
        RandomWalkWithDrift(),
        HistoricAverage(),
    ],
    freq="D",
    n_jobs=1
)

base_daily_val = sf_daily.forecast(df=daily_train, h=HORIZON_DAILY)
base_daily_test = sf_daily.forecast(df=pd.concat([daily_train, daily_val]), h=HORIZON_DAILY)

for forecast in [base_daily_val, base_daily_test]:
    forecast["Structural"] = (forecast["SeasonalNaive"] + forecast["RWD"]) / 2

base_daily_val = base_daily_val.merge(daily_val, on=["unique_id", "ds"], how="left")

base_daily_val.write_csv("../results/forecasts/base_daily_val.csv")
base_daily_test.write_csv("../results/forecasts/base_daily_test.csv")

In [ ]:
sf_monthly = StatsForecast(
    models=[
        SeasonalNaive(season_length=HORIZON_MONTHLY),
        RandomWalkWithDrift(),
        HistoricAverage(),
    ],
    freq="MS",
    n_jobs=1
)

base_monthly_val = sf_monthly.forecast(df=monthly_train, h=HORIZON_MONTHLY)
base_monthly_test = sf_monthly.forecast(df=pd.concat([monthly_train, monthly_val]), h=HORIZON_MONTHLY)

for forecast in [base_monthly_val, base_monthly_test]:
    forecast["Structural"] = (forecast["SeasonalNaive"] + forecast["RWD"]) / 2

base_monthly_val = base_monthly_val.merge(monthly_val, on=["unique_id", "ds"], how="left")

base_monthly_val.write_csv("../results/forecasts/base_monthly_val.csv")
base_monthly_test.write_csv("../results/forecasts/base_monthly_test.csv")

In [32]:
base_monthly_val

,unique_id,ds,SeasonalNaive,RWD,HistoricAverage,Structural,y
0,Austria,2025-01-01,80.597097,128.681597,76.286243,104.639347,134.178710
1,Austria,2025-02-01,65.414483,129.504484,76.286243,97.459483,140.771071
2,Austria,2025-03-01,62.858710,130.327371,76.286243,96.593040,103.604194
3,Austria,2025-04-01,58.865000,131.150258,76.286243,95.007629,81.323667
4,Austria,2025-05-01,64.170645,131.973144,76.286243,98.071895,71.016774
5,Austria,2025-06-01,68.320000,132.796031,76.286243,100.558016,66.672667
6,Austria,2025-07-01,63.659355,133.618918,76.286243,98.639137,87.980000
7,Austria,2025-08-01,84.477097,134.441805,76.286243,109.459451,74.744194
8,Austria,2025-09-01,82.400667,135.264692,76.286243,108.832679,92.587667
9,Austria,2025-10-01,85.353548,136.087579,76.286243,110.720564,108.226452


##### Statistical Forecast:

In [ ]:
sf_daily = StatsForecast(
    models=[
        SimpleExponentialSmoothing(alpha=0.5),
        Holt(season_length=HORIZON_DAILY, error_type="A"),
        HoltWinters(season_length=HORIZON_DAILY, error_type="A"),
        AutoRegressive(lags=HORIZON_DAILY),
        AutoARIMA()
    ],
    freq="D",
    n_jobs=1
)

stat_daily_val = sf_daily.forecast(df=daily_train, h=HORIZON_DAILY)
stat_daily_test = sf_daily.forecast(df=pd.concat([daily_train, daily_val]), h=HORIZON_DAILY)

stat_daily_val = stat_daily_val.merge(daily_val, on=["unique_id", "ds"], how="left")

stat_daily_val.write_csv("../results/forecasts/stat_daily_val.csv")
stat_daily_test.write_csv("../results/forecasts/stat_daily_test.csv")

In [ ]:
sf_monthly = StatsForecast(
    models=[
        SimpleExponentialSmoothing(alpha=0.5),
        Holt(season_length=HORIZON_MONTHLY, error_type="A"),
        HoltWinters(season_length=HORIZON_MONTHLY, error_type="A"),
        AutoRegressive(lags=HORIZON_MONTHLY),
        AutoARIMA()
    ],
    freq="MS",
    n_jobs=1
)

stat_monthly_val = sf_monthly.forecast(df=monthly_train, h=HORIZON_MONTHLY)
stat_monthly_test = sf_monthly.forecast(df=pd.concat([monthly_train, monthly_val]), h=HORIZON_MONTHLY)

stat_monthly_val = stat_monthly_val.merge(monthly_val, on=["unique_id", "ds"], how="left")

stat_monthly_val.write_csv("../results/forecasts/stat_monthly_val.csv")
stat_monthly_test.write_csv("../results/forecasts/stat_monthly_test.csv")